# EBay Car Sales Data

We'll work with a dataset of used cars from eBay Kleinanzeigen, a classifieds section of the German eBay website. We sampled 50,000 data points from the full dataset to be able to run it locally without any issues. The original dataset can be found [here](https://www.kaggle.com/orgesleka/used-cars-database/data).

The data dictionary provided with data is as follows:
- `dateCrawled`: When this ad was first crawled. All field-values are taken from this date.
- `name`: Name of the car.
- `seller`: Whether the seller is private or a dealer.
- `offerType`: The type of listing
- `price`: The price on the ad to sell the car.
- `abtest`: Whether the listing is included in an A/B test.
- `vehicleType`: The vehicle Type.
- `yearOfRegistration`: The year in which the car was first registered.
- `gearbox`: The transmission type.
- `powerPS`: The power of the car in PS.
- `model`: The car model name.
- `kilometer`: How many kilometers the car has driven.
- `monthOfRegistration`: The month in which the car was first registered.
- `fuelType`: What type of fuel the car uses.
- `brand`: The brand of the car.
- `notRepairedDamage`: If the car has a damage which is not yet repaired.
- `dateCreated`: The date on which the eBay listing was created.
- `nrOfPictures`: The number of pictures in the ad.
- `postalCode`: The postal code for the location of the vehicle.
- `lastSeenOnline`: When the crawler saw this ad last online.

The aim of this project is to clean the data and analyze the included used car listings.

In [2]:
import numpy as np
import pandas as pd

In [4]:
autos = pd.read_csv('autos.csv', encoding='Latin-1')

In [6]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [7]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

We notice the following after looking at the first few rows and info of the dataframe:
- `dateCrawled`: should be of type datetime.
- `name`: needs to be cleaned from all of the extra characters and should probably be split into multiple fields
- `price`: should be changed to a `float` type after stripping all the special characters. Then possibly changing the volumn name itself.
- `vehicleType`: is missing some data.
- `yearOfRegistration`: can be switched to type `date` or can be left the way it is.
- `gearbox`: has some missing values.
- `model`: has some missing values.
- `odometer`: needs to be striped of the special characters and changed to a `float` type. Once that's done the column name needs to change.
- `fuelType`: has some missing values.
- `notRepairedDamage`: has some missing values and can probably be changed to a `bool` type.
- `dateCreated`: can be changed to be of type date.
- `lastSeen`: can be changed to be of type date.

# Cleaning Column Names

We need to change the names of some fields in order to clarify what the fields are for. We'll also changed the field names from `Camel Case` to `Snake Case` because that's the convention for `Python`. Also if you use `Camel Case` with `Python` you're a bad boy/girl.

In [8]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [9]:
new_cols = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
       'last_seen']
autos.columns = new_cols

In [11]:
autos.columns

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
       'last_seen'],
      dtype='object')

# Initial Exploration

We'll use the `describe()` to further explor our data and look at descriptive statistics for all columns.

In [12]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-29 23:42:13,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


Any columns that have mostly one value that are candidates to be dropped?
- `nr_of_pictures` has the value 0 for all rows. It can be dropped or ignored.

Any columns that need more investigation?
- `fuel_type`, `odometer`, `gearbox`, `registration_year`, `vehicle_type`, `price`.

Any examples of numeric data stored as text that needs to be cleaned?
- `price`, `odometer`

# Cleaning the `price` and `odometer` columns

For the `price` and `odometer` columns we'll remove the characters and convert them to numeric columns. We'll then rename the columns to include the unit as part of the column name.

In [13]:
autos['odometer'] = (autos['odometer']
                     .str.replace(',', '')
                     .str.replace('km', '')
                     .astype(int)
                    )
autos.rename({'odometer':'odometer_km'}, axis=1, inplace=True)
autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,150000,3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,150000,6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,70000,7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,70000,6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,150000,7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [14]:
autos['price'] = (autos['price']
                     .str.replace(',', '')
                     .str.replace('$', '')
                     .astype(float)
                    )
autos.rename({'price':'price_dollar'}, axis=1, inplace=True)
autos.head()

,date_crawled,name,seller,offer_type,price_dollar,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,5000.0,control,bus,2004,manuell,158,andere,150000,3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,8500.0,control,limousine,1997,automatik,286,7er,150000,6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,8990.0,test,limousine,2009,manuell,102,golf,70000,7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,4350.0,control,kleinwagen,2007,automatik,71,fortwo,70000,6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,1350.0,test,kombi,2003,manuell,0,focus,150000,7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [8]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
date_crawled          50000 non-null object
name                  50000 non-null object
seller                50000 non-null object
offer_type            50000 non-null object
price_dollar          50000 non-null float64
abtest                50000 non-null object
vehicle_type          44905 non-null object
registration_year     50000 non-null int64
gearbox               47320 non-null object
power_ps              50000 non-null int64
model                 47242 non-null object
odometer_km           50000 non-null int64
registration_month    50000 non-null int64
fuel_type             45518 non-null object
brand                 50000 non-null object
unrepaired_damage     40171 non-null object
ad_created            50000 non-null object
nr_of_pictures        50000 non-null int64
postal_code           50000 non-null int64
last_seen             50000 non-null object
dtypes: float64(

# Exploring `price` and `odometer`

We'll continue to inspect `price` and `odometer` to see if there are any outliers that need to be removed.

In [15]:
autos['odometer_km'].unique()

array([150000,  70000,  50000,  80000,  10000,  30000, 125000,  90000,
        20000,  60000,   5000, 100000,  40000])

We can see that the values rounded, which probably means that sellers were given choices to select from.

In [16]:
autos['odometer_km'].describe()

count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

In [18]:
autos['odometer_km'].value_counts().sort_index(ascending=True)

5000        967
10000       264
20000       784
30000       789
40000       819
50000      1027
60000      1164
70000      1230
80000      1436
90000      1757
100000     2169
125000     5170
150000    32424
Name: odometer_km, dtype: int64

In [20]:
autos['price_dollar'].unique()

array([ 5000.,  8500.,  8990., ...,   385., 22200., 16995.])

In [21]:
autos['price_dollar'].unique().shape

(2357,)

In [22]:
autos['price_dollar'].describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price_dollar, dtype: float64

In [26]:
autos['price_dollar'].value_counts().sort_index(ascending=True).head(20)

0.0     1421
1.0      156
2.0        3
3.0        1
5.0        2
8.0        1
9.0        1
10.0       7
11.0       2
12.0       3
13.0       2
14.0       1
15.0       2
17.0       3
18.0       1
20.0       4
25.0       5
29.0       1
30.0       7
35.0       1
Name: price_dollar, dtype: int64

In [25]:
autos['price_dollar'].value_counts().sort_index(ascending=False).head(20)

99999999.0    1
27322222.0    1
12345678.0    3
11111111.0    2
10000000.0    1
3890000.0     1
1300000.0     1
1234566.0     1
999999.0      2
999990.0      1
350000.0      1
345000.0      1
299000.0      1
295000.0      1
265000.0      1
259000.0      1
250000.0      1
220000.0      1
198000.0      1
197000.0      1
Name: price_dollar, dtype: int64

# Removing outliers using price

There are **1421** cars priced at **0 dollars**. Since this is only 2% of the of the data, we can consider removing these rows. There are **14** cars priced at around or over **1 million dollars**.

Given that eBay is an auction site, there could be items with an opening bid of 1 dollar. So we'll keep items priced between 1 dollar and 351000 dollars because after this number there was an unrealistic jump.

In [27]:
autos = autos[autos["price_dollar"].between(1,351000)]

# Exploring and Cleaning the Dates

In [29]:
autos['date_crawled'].str[:10].value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.027907
2016-03-06    0.013712
2016-03-07    0.034733
2016-03-08    0.031350
2016-03-09    0.031471
2016-03-10    0.032679
2016-03-11    0.033162
2016-03-12    0.038115
2016-03-13    0.016611
2016-03-14    0.036786
2016-03-15    0.032317
2016-03-16    0.028692
2016-03-17    0.030082
2016-03-18    0.013047
2016-03-19    0.038478
2016-03-20    0.038538
2016-03-21    0.035156
2016-03-22    0.032075
2016-03-23    0.030686
2016-03-24    0.030142
2016-03-25    0.031169
2016-03-26    0.032619
2016-03-27    0.031954
2016-03-28    0.034733
2016-03-29    0.035760
2016-03-30    0.033525
2016-03-31    0.031592
2016-04-01    0.035699
2016-04-02    0.036726
2016-04-03    0.040471
2016-04-04    0.033887
2016-04-05    0.012202
2016-04-06    0.002175
2016-04-07    0.001752
Name: date_crawled, dtype: float64

Seems like a normal distribution, but the last two days are pretty low.

In [18]:
autos['ad_created'].str[:10].value_counts(normalize=True, dropna=False).sort_index()

2015-06-11    0.000058
2015-08-10    0.000058
2015-09-09    0.000058
2015-11-10    0.000058
2015-12-30    0.000058
2016-01-03    0.000058
2016-01-10    0.000058
2016-01-22    0.000058
2016-01-27    0.000117
2016-02-01    0.000058
2016-02-05    0.000058
2016-02-07    0.000058
2016-02-09    0.000058
2016-02-14    0.000058
2016-02-16    0.000058
2016-02-17    0.000058
2016-02-18    0.000058
2016-02-19    0.000117
2016-02-20    0.000058
2016-02-21    0.000117
2016-02-23    0.000234
2016-02-24    0.000058
2016-02-25    0.000175
2016-02-26    0.000058
2016-02-27    0.000117
2016-02-28    0.000351
2016-02-29    0.000234
2016-03-01    0.000058
2016-03-02    0.000175
2016-03-03    0.000760
                ...   
2016-03-09    0.031915
2016-03-10    0.032032
2016-03-11    0.033376
2016-03-12    0.038052
2016-03-13    0.017886
2016-03-14    0.035480
2016-03-15    0.032441
2016-03-16    0.029577
2016-03-17    0.029635
2016-03-18    0.013912
2016-03-19    0.036942
2016-03-20    0.038403
2016-03-21 

Pretty stable between months 6 2015 to 3 2016 with sudden increases, however there is no data for months 7 and 10 of 2015. The values increase alot in month 3 then start to decrease towards the beginning of month 4.

In [19]:
autos['last_seen'].str[:10].value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.000994
2016-03-06    0.003507
2016-03-07    0.004092
2016-03-08    0.004793
2016-03-09    0.007365
2016-03-10    0.009352
2016-03-11    0.009586
2016-03-12    0.021686
2016-03-13    0.006956
2016-03-14    0.010288
2016-03-15    0.014671
2016-03-16    0.013327
2016-03-17    0.023790
2016-03-18    0.007248
2016-03-19    0.013561
2016-03-20    0.018062
2016-03-21    0.017828
2016-03-22    0.019114
2016-03-23    0.016776
2016-03-24    0.017010
2016-03-25    0.017419
2016-03-26    0.014964
2016-03-27    0.013912
2016-03-28    0.018646
2016-03-29    0.021744
2016-03-30    0.022855
2016-03-31    0.021744
2016-04-01    0.021803
2016-04-02    0.024082
2016-04-03    0.023848
2016-04-04    0.021803
2016-04-05    0.138649
2016-04-06    0.247837
2016-04-07    0.150690
Name: last_seen, dtype: float64

The data starts low and then keeps increasing with sudden low points in between. However overall it keeps increasing.

In [20]:
autos['registration_year'].describe()

count    17108.000000
mean      2009.462824
std        148.974562
min       1000.000000
25%       2002.000000
50%       2008.000000
75%       2011.000000
max       9999.000000
Name: registration_year, dtype: float64

The min is incorrect because that is before cars were created and the max is incorrect as well because the year is in the future.

In [21]:
autos['registration_year'].between(1900, 2016).sum()

16555

In [30]:
autos = autos[autos['registration_year'].between(1900, 2016)]
autos['registration_year'].value_counts()

2009    1387
2011    1313
2012    1176
2010    1165
2008    1150
2006     934
2007     920
2005     842
2013     757
2014     659
2004     656
2003     633
2002     540
2000     537
2001     505
1999     447
2015     388
1998     379
1997     292
2016     291
1995     237
1996     220
1994      99
1990      98
1992      92
1993      90
1991      75
1980      56
1989      50
1985      43
        ... 
1966      15
1977      13
1965      13
1976      12
1964      12
1975      12
1973      12
1959       4
1961       4
1956       4
1937       3
1958       3
1962       3
1910       3
1963       3
1941       2
1955       2
1951       2
1950       2
1954       2
1957       2
1934       2
1948       1
1931       1
1927       1
1929       1
1943       1
1938       1
1939       1
1952       1
Name: registration_year, Length: 77, dtype: int64

The newer the car, the more registered it is. However, cars that are very recent are registered less probably due to their high prices.

The increase in registration started to happen in the mid 60s.

# Exploring Brands

In [32]:
brands = (autos['brand'].value_counts(normalize=True)[:6].index)
brands

Index(['volkswagen', 'opel', 'bmw', 'mercedes_benz', 'audi', 'ford'], dtype='object')

Chose the top brands, where each brand coveres at least over 5 percent of all cars.

In [33]:
brand_dict = {}
for brand in brands:
    mean_price = autos.loc[autos['brand']==brand, 'price_dollar'].mean()
    brand_dict[brand] = mean_price

brand_dict

{'volkswagen': 5332.4784249226,
 'opel': 2944.6075421641085,
 'bmw': 8261.382442169132,
 'mercedes_benz': 8536.027085124677,
 'audi': 9212.9306621881,
 'ford': 3728.4121821407452}

In [34]:
mean_kms = {}
for brand in brands:
    mean_km = autos.loc[autos['brand']==brand, 'odometer_km'].mean()
    mean_kms[brand] = mean_km

mean_kms

{'volkswagen': 128896.57507739938,
 'opel': 129383.17225696419,
 'bmw': 132682.97307546454,
 'mercedes_benz': 130796.43164230438,
 'audi': 129492.56238003839,
 'ford': 124349.49733885274}

In [35]:
bmp_series = pd.Series(brand_dict)
bmkm_series = pd.Series(mean_kms)

df = pd.DataFrame(bmp_series, columns=['mean_price'])
df['mean_kms'] = bmkm_series
df

,mean_price,mean_kms
volkswagen,5332.478425,128896.575077
opel,2944.607542,129383.172257
bmw,8261.382442,132682.973075
mercedes_benz,8536.027085,130796.431642
audi,9212.930662,129492.562380
ford,3728.412182,124349.497339


- The top two commen cars are the ones with the lower average price, the others have higher prices.
- Mean Kms doesn't vary a lot and the averages are considered pretty close.
- I think it's safe to say the deciding factor is pretty much the price and the kms have low impact.